<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#1.-Setting-Parameter" data-toc-modified-id="1.-Setting-Parameter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Setting Parameter</a></span></li><li><span><a href="#Data-Collect" data-toc-modified-id="Data-Collect-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Collect</a></span></li><li><span><a href="#3.-Example-Input-Data" data-toc-modified-id="3.-Example-Input-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Example Input Data</a></span></li><li><span><a href="#4.-Example-Save-Data" data-toc-modified-id="4.-Example-Save-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Example Save Data</a></span></li></ul></div>

# Import

In [1]:
import pandas as pd
import os
import sys

sys.path.append("../")
sys.path.append("../..")

from Clust.setting.influx_setting_KETI import CLUSTDataServer2 as ins
from Clust.clust.ingestion.influx import influx_Client_v2 as iC

import Clust.clust.ingestion.CSVtoInflux.csvCollector as cC

# 1. Setting Parameter

- parameter 는 KETIPreDataCollection.collectorParameter 의 parameter 로 호출 가능
```
###예시###
import KETIPreDataCollection.collectorParameter as cP
cP.parameter
```

In [2]:
####### db connection parameter
db_client = iC.influxClient(ins)

In [3]:
####### Folder / File 업로드 방식 고르시
upload_type_list = ["File", "Folder"]
upload_type = upload_type_list[0]

In [4]:
####### read&clean type 고르기 parameter
data_read_type_list = ["basic", "selectedData", "highCapacity"]
data_read_type = data_read_type_list[1]

In [5]:
####### 기본 Data 관련 저장에 필요한 정보 기입
db_name = "test_data_type3"
ms_name = "test3"
path = "./Data Collection_Test Data Folder/TimeDuplicatoinData_Type3.csv"
encoding = "cp949"

In [6]:
####### 저장될 데이터의 시간 스템프로 지정될 column 을 기입하는 parameter
time_column = "사용일자"

In [7]:
####### clean type 에 따라 data read type의 parameter 가 달라짐
selected_datas =  [
    {"Selected_columns":["노선번호", "역명"]},
    {"Selected_values":["100", "한성여객종점"]},
    {"Selected_Function":["Equal", "Equal"]}
]

In [8]:
####### 저장하는 column 선택하는 parameter
selected_columns = [{"Selected_columns":['승차총승객수', '하차총승객수']},
                    {"Rename_columns":['number of passengers getting in', 'number of passengers getting off']}]

In [9]:
####### duplicated time column processing method 정하는 parameter
dtcpm = [
    {"Selected_columns":['number of passengers getting in', 'number of passengers getting off']},
    {"Processing_method":["Max", "Min"]}
]

In [10]:
####### 최종 Parameter
parameter = {
    "uploadType" : upload_type,
    "dataReadType" : data_read_type,
    "dbName" : db_name,
    "msName" : ms_name,
    "filePath" : path,
    "selectedDatas" : selected_datas, # data read type 이 selectedData 인 경우 필요한 param
    "timeColumnName" : time_column,
    "selectedColumns" : selected_columns,
    "duplicatedTimeColumnProcessingMethod" : dtcpm,
    "encoding" : encoding
}
print("parameter success")

parameter success


# Data Collect

In [11]:
csv_collector = cC.Collector(parameter, db_client)
csv_collector.collect()

===========data clean success===========
Writing Data ...
=========== ms name : test3 ===========
            number of passengers getting in  number of passengers getting off
time                                                                         
2021-04-29                               16                                 1
2021-04-30                               12                                 1
========== write success ==========


# 3. Example Input Data

In [12]:
ex_data = pd.read_csv(path, encoding = 'cp949')

In [13]:
ex_data.head()

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20210401,100,100번(하계동~용산구청),11428,한성여객종점,19,2,20210404
1,20210401,100,100번(하계동~용산구청),11374,서울시립북서울미술관,48,0,20210404
2,20210401,100,100번(하계동~용산구청),11373,중계역,51,3,20210404
3,20210401,100,100번(하계동~용산구청),11434,골마을근린공원,318,89,20210404
4,20210401,100,100번(하계동~용산구청),08007,돈암사거리.성신여대입구,105,361,20210404


# 4. Example Save Data 

In [14]:
db_client = iC.influxClient(ins)
save_data = db_client.get_data(db_name, ms_name)

In [15]:
save_data.head()

,number of passengers getting in,number of passengers getting off
time,,
2021-04-01 00:00:00+00:00,19,2
2021-04-02 00:00:00+00:00,14,5
2021-04-03 00:00:00+00:00,5,2
2021-04-04 00:00:00+00:00,4,0
2021-04-05 00:00:00+00:00,24,1
